In [2]:
import os
# Change this if you don't want the data to be extracted in the current directory.
data_dir = '.'
if not os.path.exists(data_dir):
  os.makedirs(data_dir)

In [3]:
import os
import numpy as np
import pickle
import pandas as pd
import json
import re
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Audio
from IPython.display import display
from scipy.io import wavfile
from itertools import chain 
import string
import Levenshtein as Lev
from itertools import groupby
import scipy.stats as st
from scipy import signal
import nltk
from scipy.stats import wasserstein_distance as wd
from sklearn.metrics.pairwise import euclidean_distances as ed
from pyemd import emd
from hyphenate import hyphenate_word
from itertools import islice 
from collections import OrderedDict
import csv

In [4]:
#####################################################################################
########### set the Data files paths on Conda Notebook on Ubuntu #################### 

### 
os.chdir('/home/mmm2050/QU_DFKI_Thesis/Experimentation/ASR_Accent_Analysis_De')
main_path=os.getcwd()

if not os.path.exists(main_path+'/Data_results'):
  os.makedirs(main_path+'/Data_results')

if not os.path.exists(main_path+'/Figures_results'):
  os.makedirs(main_path+'/Figures_results')

Data_path=main_path+'/Data/'
Data_results_path=main_path+'/Data_results/'
json_file_path=Data_path+'results.json'
json_file_path_small=Data_path+'results_small.json'
validated_tsv_path=Data_path+'validated.tsv'
validated_tsv_path_small=Data_path+'validated_small.tsv'
 

validated_en_tsv_path=main_path+'/DeepSpeech/data/validated.tsv'
validated_en_tsv_path_small=main_path+'/DeepSpeech/data/validated_small.tsv'

 
os.getcwd()

'/home/mmm2050/QU_DFKI_Thesis/Experimentation/ASR_Accent_Analysis_De'

### **Show `Audio file `and `Accent` of an given `Word`**:

In [5]:
#@title ###**According to a given word, this code shows and writes in which Audio files and which Accent this given word is located:**

###########################################################################################################################################################
########### creat a Dataframe called "dataset_audiofilename_transcript_accent" contains all audiofilename and thier 'transcript' and 'test_file' /Accent####
###########################################################################################################################################################
list_dataset_test_=['test_at.txt','test_gb.txt','test_it.txt','test_de_al.txt','test_fr.txt','test_de_ni.txt','test_ch.txt',
               'test_de.txt','test_us.txt','test_ca.txt','test_ru.txt']
list_dataset_test_accent=['Österreichisches Deutsch','Britisches Deutsch','Italienisch Deutsch','Alemannische Färbung,Schweizer Standart Deutsch',
'Französisch Deutsch','Niederländisch Deutsch','Schweizerdeutsch','Deutschland Deutsch','Amerikanisches Deutsch','Kanadisches Deutsch','Russisch Deutsch']

data_dict_accent_duration  = dict(zip( list_dataset_test_, list_dataset_test_accent))  


### here is the required_word
required_word="das"




newfile=0
# dict_audiofilename_transcript_accent=[]
# To find out the Audio file's Accent
list_test_accent_txt_values=[]
list_test_accent_txt_keys=[]
list_test_file=[]
list_accent_long=[]

if __name__ == "__main__":
  Dict_results = json.load(open(json_file_path))
  for test_file in Dict_results:
    # print(test_file)
    list_test_accent_txt_values_temp = [v["reference"] for v in Dict_results[test_file].values()]
    list_test_accent_txt_keys_temp=[re.split(r'[.|/]',k)[9] for k in Dict_results[test_file].keys()]
    list_test_file_temp=[test_file for v in Dict_results[test_file].values()]

# creat a Dataframe called  dataset_audiofilename_transcript_accent contains all audiofilename and thier 'transcript' and 'test_file' /Accent
    list_test_accent_txt_values.extend(list_test_accent_txt_values_temp)
    list_test_accent_txt_keys.extend(list_test_accent_txt_keys_temp)
    list_test_file.extend(list_test_file_temp)
    keys=['audiofilename','transcript','test_file']
    trans_dict_test_file_result = dict(zip(keys,[list_test_accent_txt_keys, list_test_accent_txt_values,list_test_file]))
    
dataset_audiofilename_transcript_accent = pd.DataFrame(trans_dict_test_file_result)

#############################################################################################
##### Show and write which Audio files and which Accent the given word is located ###########

for key, value in dataset_audiofilename_transcript_accent['transcript'].items():
  if required_word in value:

# to show the full name accent of the founded result 
    for keys_accent_long_i, values_accent_long_i in data_dict_accent_duration.items():
      if dataset_audiofilename_transcript_accent.test_file[key]==keys_accent_long_i:
        print()
        # print(dataset_audiofilename_transcript_accent.audiofilename[key],dataset_audiofilename_transcript_accent.transcript[key],
        # f'*** Accent ={dataset_audiofilename_transcript_accent.test_file[key]}')
        print(dataset_audiofilename_transcript_accent.audiofilename[key],dataset_audiofilename_transcript_accent.transcript[key],
              f'*** Accent is {data_dict_accent_duration[keys_accent_long_i]}')
 

          ## Alternative way (for speed performance) to export the Audio files numbers and thiers Acccent to an external file
        if newfile==0:
          dataset_accent_write_file = open(Data_results_path+f'Audiofilesnumber_Accent_{required_word}.csv','w')
          newfile=1
        else:
          dataset_accent_write_file = open(Data_results_path+f'Audiofilesnumber_Accent_{required_word}.csv','a')
          # dataset_accent_write_file.write(f'{key}the Audio file\'s Accent file is {test_file}\n')
          dataset_accent_write_file.write(f' {dataset_audiofilename_transcript_accent.audiofilename[key]} , {dataset_audiofilename_transcript_accent.transcript[key]} , Accent is {data_dict_accent_duration[keys_accent_long_i]}\n')


          dataset_accent_write_file.close()
          # ! cp /content/Audiofilesnumber_Accent*.txt /content/drive/MyDrive/QU-DFKI-Thesis-ASR/Experimentation/data/test_data/


common_voice_de_23932593 auch das sollte man in diesem zusammenhang ansprechen *** Accent is Österreichisches Deutsch

common_voice_de_27184351 entscheidend ist dass eine ausnahme nicht nur zulässig ist sondern auch begründet werden muss *** Accent is Österreichisches Deutsch

common_voice_de_21905475 das ist so weil einige von uns politikern sich gänzlich ins abseits begeben *** Accent is Österreichisches Deutsch

common_voice_de_18516440 das ist ein äußerst klischeehaftes studentenfahrrad *** Accent is Österreichisches Deutsch

common_voice_de_21476762 das ist in der tat sehr wenig *** Accent is Österreichisches Deutsch

common_voice_de_18693338 dabei oxidiert das koks *** Accent is Österreichisches Deutsch

common_voice_de_18870904 was nützt einem transparenz wenn das system zu komplex ist um es zu verstehen *** Accent is Österreichisches Deutsch

common_voice_de_27137086 auch eine ausstellung über das alte handwerk im verler land findet man dort *** Accent is Österreichisches Deut